In [ ]:
"""This code generates the figure for verstical cross sectional profile
   Prepared by Rai. M 2021/16/1"""

#====== Code for vertical cros-seection plot===========================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from netCDF4 import Dataset
import wrf
from wrf import (getvar, to_np, get_cartopy,get_basemap, latlon_coords, vertcross,ALL_TIMES,cartopy_xlim, cartopy_ylim, interpline, CoordPair)
import cmaps
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import StrMethodFormatter
import matplotlib.gridspec as gridspec
import cartopy
import cartopy.crs as ccrs
import cartopy.mpl.geoaxes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from matplotlib import rcParams
import matplotlib
"""Control font"""
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.sans-serif'] = "Times New Roman"


#============== DATA - Control Experiments =============================================================
win = [Dataset("wrfout_d01_2017-01-01_00_00_00"),
      Dataset("wrfout_d01_2017-02-01_00_00_00"),
      Dataset("wrfout_d01_2017-12-01_00_00_00")]
spr = [Dataset("wrfout_d01_2017-03-01_00_00_00.nc"),
       Dataset("wrfout_d01_2017-04-01_00_00_00.nc"),
       Dataset("wrfout_d01_2017-05-01_00_00_00.nc")]
mon = [Dataset("wrfout_d01_2017-06-01_00_00_00"),
       Dataset("wrfout_d01_2017-07-01_00_00_00"),
       Dataset("wrfout_d01_2017-08-01_00_00_00")]
aut = [Dataset("wrfout_d01_2017-09-01_00_00_00"),
       Dataset("wrfout_d01_2017-10-01_00_00_00"),
       Dataset("wrfout_d01_2017-11-01_00_00_00")]


#===== add data from experiment => no BC  =============
win1 = [Dataset("wrfout_d01_2017-01-01_nobc"),
        Dataset("wrfout_d01_2017-02-01_nobc"),
        Dataset("wrfout_d01_2017-12-01_nobc")]

spr1 = [Dataset("wrfout_d01_2017-03-01_nobc"),
        Dataset("wrfout_d01_2017-04-01_nobc"),
        Dataset("wrfout_d01_2017-05-01_nobc")]

mon1 = [Dataset("wrfout_d01_2017-06-01_nobc"),
        Dataset("wrfout_d01_2017-07-01_nobc"),
        Dataset("wrfout_d01_2017-08-01_nobc")]

aut1 = [Dataset("wrfout_d01_2017-09-01_nobc"),
        Dataset("wrfout_d01_2017-10-01_nobc"),
        Dataset("wrfout_d01_2017-11-01_nobc")]


wf = Dataset("wrfout_d01_2017-01-01_00_00_00")
#============= Get model height for mass grid ==========================================
ht_win  = getvar(win, "height_agl", units="km",method='cat')[0:17,:,:]
ht_spr  = getvar(spr, "height_agl", units="km",method='cat')[0:17,:,:]
ht_mon =  getvar(mon, "height_agl", units="km",method='cat')[0:17,:,:]
ht_aut  = getvar(aut, "height_agl", units="km",method='cat')[0:17,:,:]

#============== Get PBLH from control experiment=====================================================================
pblh_win1  = getvar(win, "PBLH",timeidx=-1,method='cat')
pblh_spr1  = getvar(spr, "PBLH",timeidx=-1,method='cat')
pblh_mon1  = getvar(mon, "PBLH",timeidx=-1,method='cat')
pblh_aut1  = getvar(aut, "PBLH",timeidx=-1,method='cat')

#============== Get PBLH from sensitive experiment=====================================================================
pblh_win2  = getvar(win1, "PBLH",timeidx=-1,method='cat')
pblh_spr2  = getvar(spr1, "PBLH",timeidx=-1,method='cat')
pblh_mon2  = getvar(mon1, "PBLH",timeidx=-1,method='cat')
pblh_aut2  = getvar(aut1, "PBLH",timeidx=-1,method='cat')

#======= Convert meter to KM  ==============
#====== Control============
pbl_win = (pblh_win1)*0.001
pbl_spr = (pblh_spr1)*0.001
pbl_mon = (pblh_mon1)*0.001
pbl_aut = (pblh_aut1)*0.001

#====== Sensitivity ============
pbl_win1 = (pblh_win2)*0.001
pbl_spr1 = (pblh_spr2)*0.001
pbl_mon1 = (pblh_mon2)*0.001
pbl_aut1 = (pblh_aut2)*0.001

#==== PBL Due to BC =============
pb_win = pbl_win-pbl_win1
pb_spr = pbl_spr-pbl_spr1
pb_mon = pbl_mon-pbl_mon1
pb_aut = pbl_aut-pbl_aut1


#=========== Get Terrain Height ===========================
ter_win = getvar(win, "ter", units="km")
ter_spr = getvar(spr, "ter", units="km")
ter_mon = getvar(mon, "ter", units="km")
ter_aut = getvar(aut, "ter", units="km")
#===== PBLH height for control ========================================
pblh_win = pbl_win+ter_win
pblh_spr = pbl_spr+ter_spr
pblh_mon = pbl_mon+ter_mon
pblh_aut = pbl_aut+ter_aut

#===== PBLH height for sensitive  ========================================
#pblh_wins = pbl_win1+ter_win
#pblh_sprs = pbl_spr1+ter_spr
#pblh_mons = pbl_mon1+ter_mon
#pblh_auts = pbl_aut1+ter_aut


#=== PBLH change due to BC ========================
pbl_win_bc = pb_win+ter_win
pbl_spr_bc = pb_spr+ter_spr
pbl_mon_bc = pb_mon+ter_mon
pbl_aut_bc = pb_aut+ter_aut



#========= Get ALT =========================================
alt_win  = getvar(win, "ALT",timeidx=-1,method='cat')[0]
alt_spr  = getvar(spr, "ALT",timeidx=-1,method='cat')[0]
alt_mon  = getvar(mon, "ALT",timeidx=-1,method='cat')[0]
alt_aut  = getvar(aut, "ALT",timeidx=-1,method='cat')[0]

#======= Get dbz==========================================
dbz      = getvar(win, "dbz", timeidx=-1)
#======= Seasonal ter height =============================
win_ter      = getvar(win, "ter", units="km")
spr_ter      = getvar(spr, "ter", units="km")
mon_ter      = getvar(mon, "ter", units="km")
aut_ter      = getvar(aut, "ter", units="km")
#==== Load SA data================================
sa_win = [Dataset("wrfout_d01_2017-01-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-02-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-12-01_00_00_00-SA")]
sa_spr = [Dataset("wrfout_d01_2017-03-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-04-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-05-01_00_00_00-SA")]
sa_mon = [Dataset("wrfout_d01_2017-06-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-07-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-08-01_00_00_00-SA")]
sa_aut = [Dataset("wrfout_d01_2017-09-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-10-01_00_00_00-SA"),
          Dataset("wrfout_d01_2017-11-01_00_00_00-SA")]
#==== Load EC data================================
ec_win = [Dataset("wrfout_d01_2017-01-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-02-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-12-01_00_00_00-EC")]
ec_spr = [Dataset("wrfout_d01_2017-03-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-04-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-05-01_00_00_00-EC")]
ec_mon = [Dataset("wrfout_d01_2017-06-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-07-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-08-01_00_00_00-EC")]
ec_aut = [Dataset("wrfout_d01_2017-09-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-10-01_00_00_00-EC"),
          Dataset("wrfout_d01_2017-11-01_00_00_00-EC")]
#==== Load NC data================================
nc_win = [Dataset("wrfout_d01_2017-01-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-02-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-12-01_00_00_00-NC")]
nc_spr = [Dataset("wrfout_d01_2017-03-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-04-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-05-01_00_00_00-NC")]
nc_mon = [Dataset("wrfout_d01_2017-06-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-07-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-08-01_00_00_00-NC")]
nc_aut = [Dataset("wrfout_d01_2017-09-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-10-01_00_00_00-NC"),
          Dataset("wrfout_d01_2017-11-01_00_00_00-NC")]
#==== Load CA data================================
ca_win = [Dataset("wrfout_d01_2017-01-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-02-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-12-01_00_00_00-CA")]
ca_spr = [Dataset("wrfout_d01_2017-03-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-04-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-05-01_00_00_00-CA")]
ca_mon = [Dataset("wrfout_d01_2017-06-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-07-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-08-01_00_00_00-CA")]
ca_aut = [Dataset("wrfout_d01_2017-09-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-10-01_00_00_00-CA"),
          Dataset("wrfout_d01_2017-11-01_00_00_00-CA")]

#==== Get BC1 and BC2 data ======= Control case ==================================
win_bc1 = getvar(win, "BC1", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
win_bc2 = getvar(win, "BC2", timeidx=-1,method='cat')[0:17,:,:]
spr_bc1 = getvar(spr, "BC1", timeidx=-1,method='cat')[0:17,:,:]
spr_bc2 = getvar(spr, "BC2", timeidx=-1,method='cat')[0:17,:,:]
mon_bc1 = getvar(mon, "BC1", timeidx=-1,method='cat')[0:17,:,:]
mon_bc2 = getvar(mon, "BC2", timeidx=-1,method='cat')[0:17,:,:]
aut_bc1 = getvar(aut, "BC1", timeidx=-1,method='cat')[0:17,:,:]
aut_bc2 = getvar(aut, "BC2", timeidx=-1,method='cat')[0:17,:,:]
#============================== SA ============================================================================
sa_win_bc1 = getvar(sa_win, "BC1", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
sa_win_bc2 = getvar(sa_win, "BC2", timeidx=-1,method='cat')[0:17,:,:]
sa_spr_bc1 = getvar(sa_spr, "BC1", timeidx=-1,method='cat')[0:17,:,:]
sa_spr_bc2 = getvar(sa_spr, "BC2", timeidx=-1,method='cat')[0:17,:,:]
sa_mon_bc1 = getvar(sa_mon, "BC1", timeidx=-1,method='cat')[0:17,:,:]
sa_mon_bc2 = getvar(sa_mon, "BC2", timeidx=-1,method='cat')[0:17,:,:]
sa_aut_bc1 = getvar(sa_aut, "BC1", timeidx=-1,method='cat')[0:17,:,:]
sa_aut_bc2 = getvar(sa_aut, "BC2", timeidx=-1,method='cat')[0:17,:,:]
#================================ EC  ========================================================================================
ec_win_bc1 = getvar(ec_win, "BC1", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
ec_win_bc2 = getvar(ec_win, "BC2", timeidx=-1,method='cat')[0:17,:,:]
ec_spr_bc1 = getvar(ec_spr, "BC1", timeidx=-1,method='cat')[0:17,:,:]
ec_spr_bc2 = getvar(ec_spr, "BC2", timeidx=-1,method='cat')[0:17,:,:]
ec_mon_bc1 = getvar(ec_mon, "BC1", timeidx=-1,method='cat')[0:17,:,:]
ec_mon_bc2 = getvar(ec_mon, "BC2", timeidx=-1,method='cat')[0:17,:,:]
ec_aut_bc1 = getvar(ec_aut, "BC1", timeidx=-1,method='cat')[0:17,:,:]
ec_aut_bc2 = getvar(ec_aut, "BC2", timeidx=-1,method='cat')[0:17,:,:]
#============================== NC  ==========================================================================================
nc_win_bc1 = getvar(nc_win, "BC1", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
nc_win_bc2 = getvar(nc_win, "BC2", timeidx=-1,method='cat')[0:17,:,:]
nc_spr_bc1 = getvar(nc_spr, "BC1", timeidx=-1,method='cat')[0:17,:,:]
nc_spr_bc2 = getvar(nc_spr, "BC2", timeidx=-1,method='cat')[0:17,:,:]
nc_mon_bc1 = getvar(nc_mon, "BC1", timeidx=-1,method='cat')[0:17,:,:]
nc_mon_bc2 = getvar(nc_mon, "BC2", timeidx=-1,method='cat')[0:17,:,:]
nc_aut_bc1 = getvar(nc_aut, "BC1", timeidx=-1,method='cat')[0:17,:,:]
nc_aut_bc2 = getvar(nc_aut, "BC2", timeidx=-1,method='cat')[0:17,:,:]
#============================== CA  =================================================================================
ca_win_bc1 = getvar(ca_win, "BC1", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
ca_win_bc2 = getvar(ca_win, "BC2", timeidx=-1,method='cat')[0:17,:,:]
ca_spr_bc1 = getvar(ca_spr, "BC1", timeidx=-1,method='cat')[0:17,:,:]
ca_spr_bc2 = getvar(ca_spr, "BC2", timeidx=-1,method='cat')[0:17,:,:]
ca_mon_bc1 = getvar(ca_mon, "BC1", timeidx=-1,method='cat')[0:17,:,:]
ca_mon_bc2 = getvar(ca_mon, "BC2", timeidx=-1,method='cat')[0:17,:,:]
ca_aut_bc1 = getvar(ca_aut, "BC1", timeidx=-1,method='cat')[0:17,:,:]
ca_aut_bc2 = getvar(ca_aut, "BC2", timeidx=-1,method='cat')[0:17,:,:]

#========== CONTROL ==============
win_bc = (win_bc1+win_bc2)
spr_bc = (spr_bc1+spr_bc2)
mon_bc = (mon_bc1+mon_bc2)
aut_bc = (aut_bc1+aut_bc2)
#========= SA =====================
sa_win_bc = (sa_win_bc1+sa_win_bc2)
sa_spr_bc = (sa_spr_bc1+sa_spr_bc2)
sa_mon_bc = (sa_mon_bc1+sa_mon_bc2)
sa_aut_bc = (sa_aut_bc1+sa_aut_bc2)
#========== EC =======================
ec_win_bc = (ec_win_bc1+ec_win_bc2)
ec_spr_bc = (ec_spr_bc1+ec_spr_bc2)
ec_mon_bc = (ec_mon_bc1+ec_mon_bc2)
ec_aut_bc = (ec_aut_bc1+ec_aut_bc2)
#=========== NC ==========================
nc_win_bc = (nc_win_bc1+nc_win_bc2)
nc_spr_bc = (nc_spr_bc1+nc_spr_bc2)
nc_mon_bc = (nc_mon_bc1+nc_mon_bc2)
nc_aut_bc = (nc_aut_bc1+nc_aut_bc2)
#=========== CA ===========================
ca_win_bc = (ca_win_bc1+ca_win_bc2)
ca_spr_bc = (ca_spr_bc1+ca_spr_bc2)
ca_mon_bc = (ca_mon_bc1+ca_mon_bc2)
ca_aut_bc = (ca_aut_bc1+ca_aut_bc2)

#====== Now get BC for each sub-region ================
#==== SA========================
win_sa = win_bc-sa_win_bc/alt_win
spr_sa = spr_bc-sa_spr_bc/alt_spr
mon_sa = mon_bc-sa_mon_bc/alt_mon
aut_sa = aut_bc-sa_aut_bc/alt_mon
#==== EC =======================
win_ec = win_bc-ec_win_bc/alt_win
spr_ec = spr_bc-ec_spr_bc/alt_spr
mon_ec = mon_bc-ec_mon_bc/alt_mon
aut_ec = aut_bc-ec_aut_bc/alt_mon
#===== NC===================
win_nc = win_bc-nc_win_bc/alt_win
spr_nc = spr_bc-nc_spr_bc/alt_spr
mon_nc = mon_bc-nc_mon_bc/alt_mon
aut_nc = aut_bc-nc_aut_bc/alt_mon
#====== CA ====================
win_ca = win_bc-ca_win_bc/alt_win
spr_ca = spr_bc-ca_spr_bc/alt_spr
mon_ca = mon_bc-ca_mon_bc/alt_mon
aut_ca = aut_bc-ca_aut_bc/alt_mon

#======================== Get Wind DATA =====================================================================
#======= Get ua and wa = CONTROL ===============================================================
win_u = getvar(win, "ua", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
win_w = getvar(win, "wa", timeidx=-1,method='cat')[0:17,:,:]
spr_u = getvar(spr, "ua", timeidx=-1,method='cat')[0:17,:,:]
spr_w = getvar(spr, "wa", timeidx=-1,method='cat')[0:17,:,:]
mon_u = getvar(mon, "ua", timeidx=-1,method='cat')[0:17,:,:]
mon_w = getvar(mon, "wa", timeidx=-1,method='cat')[0:17,:,:]
aut_u = getvar(aut, "ua", timeidx=-1,method='cat')[0:17,:,:]
aut_w = getvar(aut, "wa", timeidx=-1,method='cat')[0:17,:,:]
#========== SA ==========================================================================
sa_win_u = getvar(sa_win, "ua", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
sa_win_w = getvar(sa_win, "wa", timeidx=-1,method='cat')[0:17,:,:]
sa_spr_u = getvar(sa_spr, "ua", timeidx=-1,method='cat')[0:17,:,:]
sa_spr_w = getvar(sa_spr, "wa", timeidx=-1,method='cat')[0:17,:,:]
sa_mon_u = getvar(sa_mon, "ua", timeidx=-1,method='cat')[0:17,:,:]
sa_mon_w = getvar(sa_mon, "wa", timeidx=-1,method='cat')[0:17,:,:]
sa_aut_u = getvar(sa_aut, "ua", timeidx=-1,method='cat')[0:17,:,:]
sa_aut_w = getvar(sa_aut, "wa", timeidx=-1,method='cat')[0:17,:,:]
#======== EC ============================================================================
ec_win_u = getvar(ec_win, "ua", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
ec_win_w = getvar(ec_win, "wa", timeidx=-1,method='cat')[0:17,:,:]
ec_spr_u = getvar(ec_spr, "ua", timeidx=-1,method='cat')[0:17,:,:]
ec_spr_w = getvar(ec_spr, "wa", timeidx=-1,method='cat')[0:17,:,:]
ec_mon_u = getvar(ec_mon, "ua", timeidx=-1,method='cat')[0:17,:,:]
ec_mon_w = getvar(ec_mon, "wa", timeidx=-1,method='cat')[0:17,:,:]
ec_aut_u = getvar(ec_aut, "ua", timeidx=-1,method='cat')[0:17,:,:]
ec_aut_w = getvar(ec_aut, "wa", timeidx=-1,method='cat')[0:17,:,:]
#======== NC =====================================================================
nc_win_u = getvar(nc_win, "ua", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
nc_win_w = getvar(nc_win, "wa", timeidx=-1,method='cat')[0:17,:,:]
nc_spr_u = getvar(nc_spr, "ua", timeidx=-1,method='cat')[0:17,:,:]
nc_spr_w = getvar(nc_spr, "wa", timeidx=-1,method='cat')[0:17,:,:]
nc_mon_u = getvar(nc_mon, "ua", timeidx=-1,method='cat')[0:17,:,:]
nc_mon_w = getvar(nc_mon, "wa", timeidx=-1,method='cat')[0:17,:,:]
nc_aut_u = getvar(nc_aut, "ua", timeidx=-1,method='cat')[0:17,:,:]
nc_aut_w = getvar(nc_aut, "wa", timeidx=-1,method='cat')[0:17,:,:]
#====== CA ==============================================================================
ca_win_u = getvar(ca_win, "ua", timeidx=-1,method='cat')[0:17,:,:] # all time,surface,all lat and lon
ca_win_w = getvar(ca_win, "wa", timeidx=-1,method='cat')[0:17,:,:]
ca_spr_u = getvar(ca_spr, "ua", timeidx=-1,method='cat')[0:17,:,:]
ca_spr_w = getvar(ca_spr, "wa", timeidx=-1,method='cat')[0:17,:,:]
ca_mon_u = getvar(ca_mon, "ua", timeidx=-1,method='cat')[0:17,:,:]
ca_mon_w = getvar(ca_mon, "wa", timeidx=-1,method='cat')[0:17,:,:]
ca_aut_u = getvar(ca_aut, "ua", timeidx=-1,method='cat')[0:17,:,:]
ca_aut_w = getvar(ca_aut, "wa", timeidx=-1,method='cat')[0:17,:,:]

#================================================ Now Plot ======================================================================================================
fig, axs = plt.subplots(4, 4,figsize=(6.2,5),dpi=300)

#============ Intrpolation data SA =================================================
#========= SA - winter =========================================
z1 = 10**(win_sa/10.) # Use linear Z for interpolation
w1 = 10**(sa_win_w/10.)
u1 = 10**(sa_win_u/10.)
#========= SA - Spring =====================================
z2 = 10**(spr_sa/10.) 
w2 = 10**(sa_spr_w/10.)
u2 = 10**(sa_spr_u/10.)
#========= SA - monsoon  =====================================
z3 = 10**(mon_sa/10.)
w3 = 10**(sa_mon_w/10.)
u3 = 10**(sa_mon_u/10.)
#========= SA  - Autumn =======================================
z4 = 10**(aut_sa/10.)
w4 = 10**(sa_aut_w/10.)
u4 = 10**(sa_aut_u/10.)

#========= EC - winter =========================================
z5 = 10**(win_ec/10.) # Use linear Z for interpolation
w5 = 10**(ec_win_w/10.)
u5 = 10**(ec_win_u/10.)
#========= EC - Spring =====================================
z6 = 10**(spr_ec/10.)
w6 = 10**(ec_spr_w/10.)
u6 = 10**(ec_spr_u/10.)
#========= EC - monsoon  =====================================
z7 = 10**(mon_ec/10.)
w7 = 10**(ec_mon_w/10.)
u7 = 10**(ec_mon_u/10.)
#========= EC  - Autumn =======================================
z8 = 10**(aut_ec/10.)
w8 = 10**(ec_aut_w/10.)
u8 = 10**(ec_aut_u/10.)

#========= NC - winter =========================================
z9 = 10**(win_nc/10.) # Use linear Z for interpolation
w9 = 10**(nc_win_w/10.)
u9 = 10**(nc_win_u/10.)
#========= NC - Spring =====================================
z10 = 10**(spr_nc/10.)
w10 = 10**(nc_spr_w/10.)
u10 = 10**(nc_spr_u/10.)
#========= NC - monsoon  =====================================
z11 = 10**(mon_nc/10.)
w11 = 10**(nc_mon_w/10.)
u11 = 10**(nc_mon_u/10.)
#========= NC  - Autumn =======================================
z12 = 10**(aut_nc/10.)
w12 = 10**(nc_aut_w/10.)
u12 = 10**(nc_aut_u/10.)

#========= CA - winter =========================================
z13 = 10**(win_ca/10.) # Use linear Z for interpolation
w13 = 10**(ca_win_w/10.)
u13 = 10**(ca_win_u/10.)
#========= CA - Spring =====================================
z14 = 10**(spr_ca/10.)
w14 = 10**(ca_spr_w/10.)
u14 = 10**(ca_spr_u/10.)
#========= CA - monsoon  =====================================
z15 = 10**(mon_ca/10.)
w15 = 10**(ca_mon_w/10.)
u15 = 10**(ca_mon_u/10.)
#========= CA  - Autumn =======================================
z16 = 10**(aut_ca/10.)
w16 = 10**(ca_aut_w/10.)
u16 = 10**(ca_aut_u/10.)


#========  Define the cross section start and end points=========
cross_start = CoordPair(lat=22, lon=70)
cross_end   = CoordPair(lat=35, lon=119)
"""Compute the vertical cross-section interpolation.
   Also, include the lat/lon points along the cross-section in the metadata by setting latlon to True"""

#======= SA-winter ==============================================================================================
z1_cross = vertcross(z1, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w1_cross = vertcross(w1, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u1_cross = vertcross(u1, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= SA-Spring ==============================================================================================
z2_cross = vertcross(z2, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w2_cross = vertcross(w2, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u2_cross = vertcross(u2, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= SA-Monsoon ==============================================================================================
z3_cross = vertcross(z3, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w3_cross = vertcross(w3, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u3_cross = vertcross(u3, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= SA-Autumn ==============================================================================================
z4_cross = vertcross(z4, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w4_cross = vertcross(w4, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u4_cross = vertcross(u4, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)

#======= EC-winter ==============================================================================================
z5_cross = vertcross(z5, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w5_cross = vertcross(w5, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u5_cross = vertcross(u5, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= EC-Spring ==============================================================================================
z6_cross = vertcross(z6, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w6_cross = vertcross(w6, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u6_cross = vertcross(u6, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= EC-Monsoon ==============================================================================================
z7_cross = vertcross(z7, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w7_cross = vertcross(w7, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u7_cross = vertcross(u7, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= EC-Autumn ==============================================================================================
z8_cross = vertcross(z8, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w8_cross = vertcross(w8, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u8_cross = vertcross(u8, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)

#======= NC-winter ==============================================================================================
z9_cross = vertcross(z9, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w9_cross = vertcross(w9, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u9_cross = vertcross(u9, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= NC-Spring ==============================================================================================
z10_cross = vertcross(z10, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w10_cross = vertcross(w10, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u10_cross = vertcross(u10, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= NC-Monsoon ==============================================================================================
z11_cross = vertcross(z11, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w11_cross = vertcross(w11, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u11_cross = vertcross(u11, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= NC-Autumn ==============================================================================================
z12_cross = vertcross(z12, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w12_cross = vertcross(w12, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u12_cross = vertcross(u12, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)

#======= CA-winter ==============================================================================================
z13_cross = vertcross(z13, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w13_cross = vertcross(w13, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u13_cross = vertcross(u13, ht_win, wrfin=win,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= CA-Spring ==============================================================================================
z14_cross = vertcross(z14, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w14_cross = vertcross(w14, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u14_cross = vertcross(u14, ht_spr, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= CA-Monsoon ==============================================================================================
z15_cross = vertcross(z15, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w15_cross = vertcross(w15, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u15_cross = vertcross(u15, ht_mon, wrfin=mon,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
#======= CA-Autumn ==============================================================================================
z16_cross = vertcross(z16, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
w16_cross = vertcross(w16, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)
u16_cross = vertcross(u16, ht_aut, wrfin=spr,start_point=cross_start,end_point=cross_end,latlon=True, meta=True)


"""#===== Convert back to dBz after interpolation =============================="""

#=== SA ===
z1_cross  = 10.0 * np.log10(z1_cross)
w1_cross  = 10.0 * np.log10(w1_cross)
u1_cross  = 10.0 * np.log10(u1_cross)

z2_cross  = 10.0 * np.log10(z2_cross)
w2_cross  = 10.0 * np.log10(w2_cross)
u2_cross  = 10.0 * np.log10(u2_cross)

z3_cross  = 10.0 * np.log10(z3_cross)
w3_cross  = 10.0 * np.log10(w3_cross)
u3_cross  = 10.0 * np.log10(u3_cross)

z4_cross  = 10.0 * np.log10(z4_cross)
w4_cross  = 10.0 * np.log10(w4_cross)
u4_cross  = 10.0 * np.log10(u4_cross)
#===== EC ===========================
z5_cross  = 10.0 * np.log10(z5_cross)
w5_cross  = 10.0 * np.log10(w5_cross)
u5_cross  = 10.0 * np.log10(u5_cross)

z6_cross  = 10.0 * np.log10(z6_cross)
w6_cross  = 10.0 * np.log10(w6_cross)
u6_cross  = 10.0 * np.log10(u6_cross)

z7_cross  = 10.0 * np.log10(z7_cross)
w7_cross  = 10.0 * np.log10(w7_cross)
u7_cross  = 10.0 * np.log10(u7_cross)

z8_cross  = 10.0 * np.log10(z8_cross)
w8_cross  = 10.0 * np.log10(w8_cross)
u8_cross  = 10.0 * np.log10(u8_cross)
#====== NC ============================
z9_cross  = 10.0 * np.log10(z9_cross)
w9_cross  = 10.0 * np.log10(w9_cross)
u9_cross  = 10.0 * np.log10(u9_cross)

z10_cross  = 10.0 * np.log10(z10_cross)
w10_cross  = 10.0 * np.log10(w10_cross)
u10_cross  = 10.0 * np.log10(u10_cross)

z11_cross  = 10.0 * np.log10(z11_cross)
w11_cross  = 10.0 * np.log10(w11_cross)
u11_cross  = 10.0 * np.log10(u11_cross)

z12_cross  = 10.0 * np.log10(z12_cross)
w12_cross  = 10.0 * np.log10(w12_cross)
u12_cross  = 10.0 * np.log10(u12_cross)
#====== CA =============================
z13_cross  = 10.0 * np.log10(z13_cross)
w13_cross  = 10.0 * np.log10(w13_cross)
u13_cross  = 10.0 * np.log10(u13_cross)

z14_cross  = 10.0 * np.log10(z14_cross)
w14_cross  = 10.0 * np.log10(w14_cross)
u14_cross  = 10.0 * np.log10(u14_cross)

z15_cross  = 10.0 * np.log10(z15_cross)
w15_cross  = 10.0 * np.log10(w15_cross)
u15_cross  = 10.0 * np.log10(u15_cross)

z16_cross  = 10.0 * np.log10(z16_cross)
w16_cross  = 10.0 * np.log10(w16_cross)
u16_cross  = 10.0 * np.log10(u16_cross)


"""Make a copy of the z cross data. Let's use regular numpy arrays for this"""
#====== SA============================================
z1_cross_filled = np.ma.copy(to_np(z1_cross))
w1_cross_filled = np.ma.copy(to_np(w1_cross))
u1_cross_filled = np.ma.copy(to_np(u1_cross))

z2_cross_filled = np.ma.copy(to_np(z2_cross))
w2_cross_filled = np.ma.copy(to_np(w2_cross))
u2_cross_filled = np.ma.copy(to_np(u2_cross))

z3_cross_filled = np.ma.copy(to_np(z3_cross))
w3_cross_filled = np.ma.copy(to_np(w3_cross))
u3_cross_filled = np.ma.copy(to_np(u3_cross))

z4_cross_filled = np.ma.copy(to_np(z4_cross))
w4_cross_filled = np.ma.copy(to_np(w4_cross))
u4_cross_filled = np.ma.copy(to_np(u4_cross))

#====== EC============================================
z5_cross_filled = np.ma.copy(to_np(z5_cross))
w5_cross_filled = np.ma.copy(to_np(w5_cross))
u5_cross_filled = np.ma.copy(to_np(u5_cross))

z6_cross_filled = np.ma.copy(to_np(z6_cross))
w6_cross_filled = np.ma.copy(to_np(w6_cross))
u6_cross_filled = np.ma.copy(to_np(u6_cross))

z7_cross_filled = np.ma.copy(to_np(z7_cross))
w7_cross_filled = np.ma.copy(to_np(w7_cross))
u7_cross_filled = np.ma.copy(to_np(u7_cross))

z8_cross_filled = np.ma.copy(to_np(z8_cross))
w8_cross_filled = np.ma.copy(to_np(w8_cross))
u8_cross_filled = np.ma.copy(to_np(u8_cross))

#====== NC ============================================
z9_cross_filled = np.ma.copy(to_np(z9_cross))
w9_cross_filled = np.ma.copy(to_np(w9_cross))
u9_cross_filled = np.ma.copy(to_np(u9_cross))

z10_cross_filled = np.ma.copy(to_np(z10_cross))
w10_cross_filled = np.ma.copy(to_np(w10_cross))
u10_cross_filled = np.ma.copy(to_np(u10_cross))

z11_cross_filled = np.ma.copy(to_np(z11_cross))
w11_cross_filled = np.ma.copy(to_np(w11_cross))
u11_cross_filled = np.ma.copy(to_np(u11_cross))

z12_cross_filled = np.ma.copy(to_np(z12_cross))
w12_cross_filled = np.ma.copy(to_np(w12_cross))
u12_cross_filled = np.ma.copy(to_np(u12_cross))

#====== CA ============================================
z13_cross_filled = np.ma.copy(to_np(z13_cross))
w13_cross_filled = np.ma.copy(to_np(w13_cross))
u13_cross_filled = np.ma.copy(to_np(u13_cross))

z14_cross_filled = np.ma.copy(to_np(z14_cross))
w14_cross_filled = np.ma.copy(to_np(w14_cross))
u14_cross_filled = np.ma.copy(to_np(u14_cross))

z15_cross_filled = np.ma.copy(to_np(z15_cross))
w15_cross_filled = np.ma.copy(to_np(w15_cross))
u15_cross_filled = np.ma.copy(to_np(u15_cross))

z16_cross_filled = np.ma.copy(to_np(z16_cross))
w16_cross_filled = np.ma.copy(to_np(w16_cross))
u16_cross_filled = np.ma.copy(to_np(u16_cross))



""" For each cross section column, find the first index with non-missing
    values and copy these to the missing elements below """

#======= SA ==================================================================
for i in range(z1_cross_filled.shape[-1]):
    column_vals = z1_cross_filled[:,i]
   #Let's find the lowest index that isn't filled. The nonzero function
   #finds all unmasked values greater than 0. Since 0 is a valid value
   #for dBZ, let's change that threshold to be -200 dBZ instead.
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z1_cross_filled[0:first_idx, i] = z1_cross_filled[first_idx, i]

for i in range(z2_cross_filled.shape[-1]):
    column_vals = z2_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z2_cross_filled[0:first_idx, i] = z2_cross_filled[first_idx, i]

for i in range(z3_cross_filled.shape[-1]):
    column_vals = z3_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z3_cross_filled[0:first_idx, i] = z3_cross_filled[first_idx, i]

for i in range(z4_cross_filled.shape[-1]):
    column_vals = z4_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z4_cross_filled[0:first_idx, i] = z4_cross_filled[first_idx, i]

#======= EC  ==================================================================
for i in range(z5_cross_filled.shape[-1]):
    column_vals = z5_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z5_cross_filled[0:first_idx, i] = z5_cross_filled[first_idx, i]

for i in range(z6_cross_filled.shape[-1]):
    column_vals = z6_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z6_cross_filled[0:first_idx, i] = z6_cross_filled[first_idx, i]

for i in range(z7_cross_filled.shape[-1]):
    column_vals = z7_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z7_cross_filled[0:first_idx, i] = z7_cross_filled[first_idx, i]

for i in range(z8_cross_filled.shape[-1]):
    column_vals = z8_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z8_cross_filled[0:first_idx, i] = z8_cross_filled[first_idx, i]

#===== NC =====================================================================
for i in range(z9_cross_filled.shape[-1]):
    column_vals = z9_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z9_cross_filled[0:first_idx, i] = z9_cross_filled[first_idx, i]

for i in range(z10_cross_filled.shape[-1]):
    column_vals = z10_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z10_cross_filled[0:first_idx, i] = z10_cross_filled[first_idx, i]

for i in range(z11_cross_filled.shape[-1]):
    column_vals = z11_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z11_cross_filled[0:first_idx, i] = z11_cross_filled[first_idx, i]

for i in range(z12_cross_filled.shape[-1]):
    column_vals = z12_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z12_cross_filled[0:first_idx, i] = z12_cross_filled[first_idx, i]

#===== CA =====================================================================
for i in range(z13_cross_filled.shape[-1]):
    column_vals = z13_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z13_cross_filled[0:first_idx, i] = z13_cross_filled[first_idx, i]

for i in range(z14_cross_filled.shape[-1]):
    column_vals = z14_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z14_cross_filled[0:first_idx, i] = z14_cross_filled[first_idx, i]

for i in range(z15_cross_filled.shape[-1]):
    column_vals = z15_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z15_cross_filled[0:first_idx, i] = z15_cross_filled[first_idx, i]

for i in range(z16_cross_filled.shape[-1]):
    column_vals = z16_cross_filled[:,i]
    first_idx = int(np.transpose((column_vals > -200).nonzero())[0])
    z16_cross_filled[0:first_idx, i] = z16_cross_filled[first_idx, i]


#===============  Get the lat/lon points =============
lats, lons = latlon_coords(dbz)
cart_proj = get_cartopy(dbz)

bc_levels = np.arange(0,2,0.01)

"""#====  Get the terrain heights along the cross section line"""
#ter_line = interpline(ter, wrfin=nc1, start_point=cross_start,end_point=cross_end)
#=== control => With all aerosols =========
pblh_line_win = interpline(pblh_win, wrfin=win, start_point=cross_start,end_point=cross_end)
pblh_line_spr = interpline(pblh_spr, wrfin=spr, start_point=cross_start,end_point=cross_end)
pblh_line_sum = interpline(pblh_mon, wrfin=mon, start_point=cross_start,end_point=cross_end)
pblh_line_aut = interpline(pblh_aut, wrfin=aut, start_point=cross_start,end_point=cross_end)

#=== sensitivity ==>> without BC ========
#pblh_line_wins = interpline(pblh_wins, wrfin=win1, start_point=cross_start,end_point=cross_end)
#pblh_line_sprs = interpline(pblh_sprs, wrfin=spr1, start_point=cross_start,end_point=cross_end)
#pblh_line_sums = interpline(pblh_mons, wrfin=mon1, start_point=cross_start,end_point=cross_end)
#pblh_line_auts = interpline(pblh_auts, wrfin=aut1, start_point=cross_start,end_point=cross_end)

#=== PBLH change due to BC =============
pblh_line_win_bc = interpline(pbl_win_bc, wrfin=win, start_point=cross_start,end_point=cross_end)
pblh_line_spr_bc = interpline(pbl_spr_bc, wrfin=spr, start_point=cross_start,end_point=cross_end)
pblh_line_mon_bc = interpline(pbl_mon_bc, wrfin=mon, start_point=cross_start,end_point=cross_end)
pblh_line_aut_bc = interpline(pbl_aut_bc, wrfin=aut, start_point=cross_start,end_point=cross_end)



#===========================================
xs1 = np.arange(0, z1_cross.shape[-1], 1)
ys1 = to_np(z1_cross.coords["vertical"])

xs2 = np.arange(0, z2_cross.shape[-1], 1)
ys2 = to_np(z2_cross.coords["vertical"])


xs3 = np.arange(0, z3_cross.shape[-1], 1)
ys3 = to_np(z3_cross.coords["vertical"])

xs4 = np.arange(0, z4_cross.shape[-1], 1)
ys4 = to_np(z4_cross.coords["vertical"])

xs5 = np.arange(0, z5_cross.shape[-1], 1)
ys5 = to_np(z5_cross.coords["vertical"])

xs6 = np.arange(0, z6_cross.shape[-1], 1)
ys6 = to_np(z6_cross.coords["vertical"])

xs7 = np.arange(0, z7_cross.shape[-1], 1)
ys7 = to_np(z7_cross.coords["vertical"])

xs8 = np.arange(0, z8_cross.shape[-1], 1)
ys8 = to_np(z8_cross.coords["vertical"])

xs9 = np.arange(0, z9_cross.shape[-1], 1)
ys9 = to_np(z9_cross.coords["vertical"])

xs10 = np.arange(0, z10_cross.shape[-1], 1)
ys10 = to_np(z10_cross.coords["vertical"])

xs11 = np.arange(0, z11_cross.shape[-1], 1)
ys11 = to_np(z11_cross.coords["vertical"])

xs12 = np.arange(0, z12_cross.shape[-1], 1)
ys12 = to_np(z12_cross.coords["vertical"])

xs13 = np.arange(0, z13_cross.shape[-1], 1)
ys13 = to_np(z13_cross.coords["vertical"])

xs14 = np.arange(0, z14_cross.shape[-1], 1)
ys14 = to_np(z14_cross.coords["vertical"])

xs15 = np.arange(0, z15_cross.shape[-1], 1)
ys15 = to_np(z15_cross.coords["vertical"])

xs16 = np.arange(0, z16_cross.shape[-1], 1)
ys16 = to_np(z16_cross.coords["vertical"])


"""======Plot BC data =================================================="""
#=== SA===============
bc_contours = axs[0,0].contourf(xs1,ys1,to_np(z1_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed) 
bc_contours = axs[0,1].contourf(xs2,ys2,to_np(z2_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed) 
bc_contours = axs[0,2].contourf(xs3,ys3,to_np(z3_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed) 
bc_contours = axs[0,3].contourf(xs4,ys4,to_np(z4_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed) 
#=== EC =============
bc_contours = axs[1,0].contourf(xs5,ys5,to_np(z5_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[1,1].contourf(xs6,ys6,to_np(z6_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[1,2].contourf(xs7,ys7,to_np(z7_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[1,3].contourf(xs8,ys8,to_np(z8_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
#=== NC ============
bc_contours = axs[2,0].contourf(xs9,ys9,to_np(z9_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[2,1].contourf(xs10,ys10,to_np(z10_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[2,2].contourf(xs11,ys11,to_np(z11_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[2,3].contourf(xs12,ys12,to_np(z12_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
#=== NC ============
bc_contours = axs[3,0].contourf(xs13,ys13,to_np(z13_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[3,1].contourf(xs14,ys14,to_np(z14_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[3,2].contourf(xs15,ys15,to_np(z15_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)
bc_contours = axs[3,3].contourf(xs16,ys16,to_np(z16_cross_filled),levels=bc_levels,cmap=cmaps.WhiteBlueGreenYellowRed)

""" ====== Plot Wind data ============================================"""
#======== SA  =============================================================
axs[0,0].quiver(xs1[::4], ys1[::4],to_np(u1_cross_filled[::4, ::4]), to_np(w1_cross_filled[::4, ::4]*500))
axs[0,1].quiver(xs2[::4], ys2[::4],to_np(u2_cross_filled[::4, ::4]), to_np(w2_cross_filled[::4, ::4]*500))
axs[0,2].quiver(xs3[::4], ys3[::4],to_np(u3_cross_filled[::4, ::4]), to_np(w3_cross_filled[::4, ::4]*500))
axs[0,3].quiver(xs4[::4], ys4[::4],to_np(u4_cross_filled[::4, ::4]), to_np(w4_cross_filled[::4, ::4]*500))
#======== EC  =============================================================
axs[1,0].quiver(xs5[::4], ys5[::4],to_np(u5_cross_filled[::4, ::4]), to_np(w5_cross_filled[::4, ::4]*500))
axs[1,1].quiver(xs6[::4], ys6[::4],to_np(u6_cross_filled[::4, ::4]), to_np(w6_cross_filled[::4, ::4]*500))
axs[1,2].quiver(xs6[::4], ys7[::4],to_np(u7_cross_filled[::4, ::4]), to_np(w7_cross_filled[::4, ::4]*500))
axs[1,3].quiver(xs8[::4], ys8[::4],to_np(u8_cross_filled[::4, ::4]), to_np(w8_cross_filled[::4, ::4]*500))
#======== NC  =============================================================
axs[2,0].quiver(xs9[::4],  ys9[::4],to_np(u9_cross_filled[::4, ::4]), to_np(w9_cross_filled[::4, ::4]*500))
axs[2,1].quiver(xs10[::4], ys10[::4],to_np(u10_cross_filled[::4, ::4]), to_np(w10_cross_filled[::4, ::4]*500))
axs[2,2].quiver(xs11[::4], ys11[::4],to_np(u11_cross_filled[::4, ::4]), to_np(w11_cross_filled[::4, ::4]*500))
axs[2,3].quiver(xs12[::4], ys12[::4],to_np(u12_cross_filled[::4, ::4]), to_np(w12_cross_filled[::4, ::4]*500))
#======== CA  =============================================================
axs[3,0].quiver(xs13[::4], ys13[::4],to_np(u13_cross_filled[::4, ::4]), to_np(w13_cross_filled[::4, ::4]*500))
axs[3,1].quiver(xs14[::4], ys14[::4],to_np(u14_cross_filled[::4, ::4]), to_np(w14_cross_filled[::4, ::4]*500))
axs[3,2].quiver(xs15[::4], ys15[::4],to_np(u15_cross_filled[::4, ::4]), to_np(w15_cross_filled[::4, ::4]*500))
axs[3,3].quiver(xs16[::4], ys16[::4],to_np(u16_cross_filled[::4, ::4]), to_np(w16_cross_filled[::4, ::4]*500))


#====  Get the terrain heights along the cross section line
ter_line_win = interpline(win_ter, wrfin=win1, start_point=cross_start,end_point=cross_end)
ter_line_spr = interpline(spr_ter, wrfin=spr1, start_point=cross_start,end_point=cross_end)
ter_line_mon = interpline(mon_ter, wrfin=mon1, start_point=cross_start,end_point=cross_end)
ter_line_aut = interpline(aut_ter, wrfin=aut1, start_point=cross_start,end_point=cross_end)

"""#=========  Fill in the mountain area ========================================="""
#==== SA ===============================
axs[0,0].fill_between(xs1, 0, to_np(ter_line_win),facecolor="#5C5C5C")
axs[0,1].fill_between(xs2, 0, to_np(ter_line_spr),facecolor="#5C5C5C")
axs[0,2].fill_between(xs3, 0, to_np(ter_line_mon),facecolor="#5C5C5C")
axs[0,3].fill_between(xs4, 0, to_np(ter_line_aut),facecolor="#5C5C5C")
#==== EC ===============================
axs[1,0].fill_between(xs5, 0, to_np(ter_line_win),facecolor="#5C5C5C")
axs[1,1].fill_between(xs6, 0, to_np(ter_line_spr),facecolor="#5C5C5C")
axs[1,2].fill_between(xs7, 0, to_np(ter_line_mon),facecolor="#5C5C5C")
axs[1,3].fill_between(xs8, 0, to_np(ter_line_aut),facecolor="#5C5C5C")
#==== NC ===============================
axs[2,0].fill_between(xs9,  0, to_np(ter_line_win),facecolor="#5C5C5C")
axs[2,1].fill_between(xs10, 0, to_np(ter_line_spr),facecolor="#5C5C5C")
axs[2,2].fill_between(xs11, 0, to_np(ter_line_mon),facecolor="#5C5C5C")
axs[2,3].fill_between(xs12, 0, to_np(ter_line_aut),facecolor="#5C5C5C")
#==== CA ===============================
axs[3,0].fill_between(xs13, 0, to_np(ter_line_win),facecolor="#5C5C5C")
axs[3,1].fill_between(xs14, 0, to_np(ter_line_spr),facecolor="#5C5C5C")
axs[3,2].fill_between(xs15, 0, to_np(ter_line_mon),facecolor="#5C5C5C")
axs[3,3].fill_between(xs16, 0, to_np(ter_line_aut),facecolor="#5C5C5C")

"""===== Plot PBLH =============================================================="""
#===== SA ==================================
axs[0,0].plot(pblh_line_win,'tab:orange',linewidth=0.5)
#axs[0,0].plot(pblh_line_win_bc,'black',linewidth=0.5)

axs[0,1].plot(pblh_line_spr,'tab:orange',linewidth=0.5)
#axs[0,1].plot(pblh_line_spr_bc,'black',linewidth=0.5)

axs[0,2].plot(pblh_line_sum,'tab:orange',linewidth=0.5)
#axs[0,2].plot(pblh_line_mon_bc,'black',linewidth=0.5)

axs[0,3].plot(pblh_line_aut,'tab:orange',linewidth=0.5)
#axs[0,3].plot(pblh_line_aut_bc,'black',linewidth=0.5)

#===== EC ==================================
axs[1,0].plot(pblh_line_win,'tab:orange',linewidth=0.6)
axs[1,1].plot(pblh_line_spr,'tab:orange',linewidth=0.6)
axs[1,2].plot(pblh_line_sum,'tab:orange',linewidth=0.6)
axs[1,3].plot(pblh_line_aut,'tab:orange',linewidth=0.6)
#===== NC ==================================
axs[2,0].plot(pblh_line_win,'tab:orange',linewidth=0.6)
axs[2,1].plot(pblh_line_spr,'tab:orange',linewidth=0.6)
axs[2,2].plot(pblh_line_sum,'tab:orange',linewidth=0.6)
axs[2,3].plot(pblh_line_aut,'tab:orange',linewidth=0.6)
#===== CA ==================================
axs[3,0].plot(pblh_line_win,'tab:orange',linewidth=0.6)
axs[3,1].plot(pblh_line_spr,'tab:orange',linewidth=0.6)
axs[3,2].plot(pblh_line_sum,'tab:orange',linewidth=0.6)
axs[3,3].plot(pblh_line_aut,'tab:orange',linewidth=0.6)

""" Set the x-ticks to use latitude and longitude labels"""
coord_pairs = to_np(z1_cross.coords["xy_loc"])
x_ticks     = np.arange(coord_pairs.shape[0])
x_labels    = [pair.latlon_str(fmt="{:.1f}, {:.1f}")
              for pair in to_np(coord_pairs)]
#=======  Set the desired number of x ticks below
num_ticks = 6
thin = int((len(x_ticks) / num_ticks) + .8)

#===== Se tick====================
axs[0,0].set_xticklabels([])
#axs[0,1].set_yticklabels([])
axs[0,1].set_xticklabels([])
axs[0,2].set_yticklabels([])
axs[0,2].set_xticklabels([])
axs[0,3].set_yticklabels([])
axs[0,3].set_xticklabels([])

axs[1,0].set_xticklabels([])
#axs[1,0].set_yticklabels([])
axs[1,1].set_xticklabels([])
axs[1,1].set_yticklabels([])
axs[1,2].set_yticklabels([])
axs[1,2].set_xticklabels([])

axs[1,3].set_yticklabels([])
axs[1,3].set_xticklabels([])
axs[2,0].set_xticklabels([])
#axs[2,0].set_yticklabels([])
axs[2,1].set_xticklabels([])
axs[2,1].set_yticklabels([])
axs[2,2].set_yticklabels([])
axs[2,2].set_xticklabels([])

axs[2,3].set_yticklabels([])
axs[2,3].set_xticklabels([])
axs[3,0].set_xticklabels([])
#axs[3,0].set_yticklabels([])
axs[3,1].set_xticklabels([])
axs[3,1].set_yticklabels([])
axs[3,2].set_yticklabels([])
axs[3,3].set_yticklabels([])


axs[0,0].set_xticks(x_ticks[::thin])
axs[0,1].set_xticks(x_ticks[::thin])
axs[0,2].set_xticks(x_ticks[::thin])
axs[0,3].set_xticks(x_ticks[::thin])
axs[1,0].set_xticks(x_ticks[::thin])
axs[1,1].set_xticks(x_ticks[::thin])
axs[1,2].set_xticks(x_ticks[::thin])
axs[1,3].set_xticks(x_ticks[::thin])
axs[2,0].set_xticks(x_ticks[::thin])
axs[2,1].set_xticks(x_ticks[::thin])
axs[2,2].set_xticks(x_ticks[::thin])
axs[2,3].set_xticks(x_ticks[::thin])
axs[3,0].set_xticks(x_ticks[::thin])
axs[3,1].set_xticks(x_ticks[::thin])
axs[3,2].set_xticks(x_ticks[::thin])
axs[3,3].set_xticks(x_ticks[::thin])


#===== Set xtick label =============================================
axs[3,0].set_xticklabels(x_labels[::thin],rotation=25, fontsize=4)
axs[3,1].set_xticklabels(x_labels[::thin],rotation=25, fontsize=4)
axs[3,2].set_xticklabels(x_labels[::thin],rotation=25, fontsize=4)
axs[3,3].set_xticklabels(x_labels[::thin],rotation=25, fontsize=4)

#====  Set ytick label =============================================
axs[0,0].yaxis.set_tick_params(labelsize=4)
axs[1,0].yaxis.set_tick_params(labelsize=4)
axs[2,0].yaxis.set_tick_params(labelsize=4)
axs[3,0].yaxis.set_tick_params(labelsize=4)


axs[0,1].set_yticklabels([])
axs[0,2].set_yticklabels([])
axs[0,3].set_yticklabels([])

#===== Colorbar ======================================================
cax = fig.add_axes([0.93,0.095,0.010,0.87]) # left, bottom, width, and height
cbar= fig.colorbar(bc_contours, ax=axs[:,:],cax=cax)
cbar.set_label(r'BC [$μg m ^{-3}$]', rotation=-270, fontsize=5,labelpad=0.1)
#cbar.set_ticks([0,0.5,1,1.5,2])
#cbar.set_ticklabels([0,0.5,1,1.5,2])
#cbar.set_clim(0,2)
cbar.set_ticks([0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2])
cbar.set_ticklabels([0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2])
cbar.ax.tick_params(labelsize=5)
#====== Add x-axis title ==========
fig.text(0.012, 0.53, 'Elevation [Km]', va='center', rotation='vertical',fontsize=5)
#==== PLot title ===========
axs[0,0].set_title('Winter',fontsize=5,pad=0)
axs[0,1].set_title('Spring',fontsize=5,pad=0)
axs[0,2].set_title('Summer',fontsize=5,pad=0)
axs[0,3].set_title('Autumn',fontsize=5,pad=0)
#==== Plot y-axis ===========
axs[0,0].set_ylabel('SA',fontsize=5)
axs[1,0].set_ylabel('EA',fontsize=5)
axs[2,0].set_ylabel('NA',fontsize=5)
axs[3,0].set_ylabel('CAME',fontsize=5)

"""#======= Add cross-sectional map ====="""
sub_axes = plt.axes([0.045, .86, 0.12, 0.105])  #left, bottom, width, height
m = Basemap(projection='ortho', lat_0=30, lon_0=86,resolution='h')
m.drawmapboundary(fill_color='#A6CAE0', linewidth=0) #A6CAE0
m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
m.drawcoastlines(linewidth=0.1, color="white")
m.drawcountries(linewidth=0.1, color="white")
# Cross-section to be plotted ==========================
startlat = 22; startlon = 70
arrlat   = 35; arrlon   = 119
m.drawgreatcircle(startlon,startlat,arrlon,arrlat, linewidth=0.6, color='orange')


#===== Adjust figure size =========================
fig.subplots_adjust(top=0.965,
                        bottom=0.095,
                        left=0.067,
                        right=0.925,
                        hspace=0.105,
                        wspace=0.05)

plt.savefig("/mnt/g/2nd_Paper/BC_vertical.png",dpi=1000)
#plt.show()
